# input length test

> input length 512 이하의 행이 몇 개나 있는지?

In [ ]:

import sys
import os

from transformers import AutoTokenizer, BartForConditionalGeneration
import json
import pandas as pd

model_name = "alaggung/bart-r3f"
max_length = 512
num_beams = 10
length_penalty = 1.2

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='../cache')
model = BartForConditionalGeneration.from_pretrained(model_name, cache_dir='../cache')
model.eval()

# 'inputs', 'labels', 'preds', 'rouge', 'bleu'
df = pd.DataFrame()

def make_data(path):
    inputs = []
    labels = []

    with open(path, "r") as f:
        data = json.load(f)

    def make_chat(inp):
        for cvt in inp['conversation']:
            chats = []
            speaker = cvt['speaker']
            utterance = cvt['utterance']
            chats.append(f"화자{speaker}: {utterance}")
        
        chat = "[BOS]" + "[SEP]".join(chats)  + "[EOS]"
        return chat

    for example in data:
        chat = make_chat(example["input"])
        
        input = tokenizer(chat, return_tensors="pt")
        if input.input_ids.size(1) > 512:
            continue

        inputs.append(input)
        labels.append(example["output"])

    return inputs, labels

inputs, labels = make_data("/mnt/c/Users/hwyew/Downloads/korean_dialogue/korean_dialog/resource/data/train.json")

In [ ]:
len(inputs)

# Hanspell 성능 테스트

대화체에서도 맞춤법 교정이 유효한가?

In [ ]:
%cd /mnt/c/Users/hwyew/Downloads/korean_dialogue/korean_dialog/py-hanspell

In [ ]:
import re
import requests

def get_passport_key():
    """네이버에서 '네이버 맞춤법 검사기' 페이지에서 passportKey를 획득

        - 네이버에서 '네이버 맞춤법 검사기'를 띄운 후 
        html에서 passportKey를 검색하면 값을 찾을 수 있다.

        - 찾은 값을 spell_checker.py 48 line에 적용한다.
    """

    url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=네이버+맞춤법+검사기"
    res = requests.get(url)

    html_text = res.text

    match = re.search(r'passportKey=([^&"}]+)', html_text)
    if match:
        passport_key = match.group(1)
        return passport_key
    else:
        return False


def fix_spell_checker_py_code(file_path, passportKey):
    print(passportKey)
    """획득한 passportkey를 spell_checker.py파일에 적용
    """
    
    pattern = r"'passportKey': '.*'"

    with open(file_path, 'r', encoding='utf-8') as input_file:
        content = input_file.read()
        modified_content = re.sub(pattern, f"'passportKey': '{passportKey}'", content)

    with open(file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(modified_content)
    
    return 


# before run
spell_checker_file_path = './hanspell/spell_checker.py'

passport_key = get_passport_key()
if passport_key:
    fix_spell_checker_py_code(spell_checker_file_path, passport_key)
else:
    print("passportKey를 찾을 수 없습니다.")

In [ ]:
from hanspell import spell_checker

result = spell_checker.check(u'나도 인제 안 미루고 맞아는 보려고. 맞아서 소용없으면 소용없는 거고 또 맞아서 좋을 수도 있는 거니까. 그래서 안 걸리면 더 좋은 거고. 가 한번 name3이가 얘기한 대로 혜택 받을 수 있으면 받아가지고 맞아보고.')
result

In [ ]:
result.checked

In [ ]:
result.original

# 반복 어구 없애기

In [ ]:
import re

# 테스트 문자열 리스트
test_strings = [
    "name1 is John",
    "name123",
    "My name1 is Jane",
    "name_underscore",
    "123name",
    "너는 유산균 아직 안 먹였으면 한번 알아봐가지고 특히 name1이 먼저 먹여. name1이가 밥도 잘 안 먹고 하니까 내가 추천해 주는 거 이로울 만한 거는 없는데 한번 알아봐봐. 있을 거야"
]

# 정규표현식 패턴r'\bname\S*'
pattern = r'name[0-9]\S*'

# 정규표현식을 사용하여 어구 찾기
matches = [re.findall(pattern, string) for string in test_strings]

# 결과 출력
for string, match in zip(test_strings, matches):
    print(f"Original string: {string}")
    print(f"Matches: {match}\n")

# Hanspell 제외 전처리 테스트

In [ ]:
import json
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import koreanize_matplotlib

def make_data(data_path):
    with open(data_path, "r") as f:
        data = json.load(f)

    text_list = []
    def make_chat(inp):
        chat = ""
        for cvt in inp['conversation']:
            chat += cvt['utterance']
        return chat

    for example in data:
        text_list.append(make_chat(example['input']))
    
    text = ' '.join(text_list)
    return text



FONT_PATH = "/mnt/c/Users/hwyew/Downloads/korean_dialogue/korean_dialog/NanumSquareNeo-cBd.ttf"

def make_cloud(text, title, top_k, max_words=100):
    # plt.rc('font', family='NanumGothicCoding')
    wordcloud = WordCloud(font_path=FONT_PATH, max_words=max_words, background_color="white").generate(text)
    
    fig, axs = plt.subplots(2, 1, figsize=(15, 15))

    top_data = dict(list(wordcloud.words_.items())[:top_k])
    labels, values = zip(*top_data.items())

    # 막대 그래프 그리기
    axs[0].bar(labels, values)
    axs[0].set_xlabel('Labels')
    axs[0].set_ylabel('Values')
    axs[0].set_title(f"max count words on {title} data (top {top_k})", fontsize=36)


    # 두 번째 그래프: 워드클라우드
    axs[1].imshow(wordcloud)
    axs[1].axis('off')
    axs[1].set_title(f"max count words on {title} data (max words = {max_words})", fontsize=36)

    # 그래프 간격 조정
    plt.tight_layout()
    plt.show()

In [ ]:
import re
import json

## Preprocess functions ##
def remove_empty_utterance(data:json):
    """
    Remove empty utterances from the data
    """
    for example in data:
        example['input']['conversation'] = [cvt for cvt in example['input']['conversation'] if cvt['utterance'] != '']
    return data


def correct_wrong_output(data:json):
    """
    Correct wrong speakers in outputs of train samples 'train-000401', 'train-000402'
    """
    data[400]['output'] = data[400]['output'].replace('SD2100504','SD2110504')
    data[401]['output'] = data[401]['output'].replace('SD2110503','SD2100503')

    return data


def file_preprocess(data:json):
    data = remove_empty_utterance(data)
    data = correct_wrong_output(data)

    return data


"""
불용어 처리
- name1, name2..
- 뒤에 물결이 붙는 경우 ("음~", "아~")
- 그, 뭐, 어, 인제, 막, 아, 음, 읍, 오, 으
- 한 글자가 두번 이상 반복되는 경우 ("또 또", "그 그")
"""

stopwords_pattern = [r'name[0-9]\S*', r'\w~', r'\b으\b', r'\b그\b', r'\b뭐\b', r'\b어\b',  r'\b인제\b', r'\b막\b', r'\b아\b', r'\b음\b', r'\b읍\b', r'\b오\b', r'\b으\b']

def remove_stopwords(text):
    for pattern in stopwords_pattern:
        text = re.sub(pattern, '', text)
    
    # 두 번 이상 반복되는 경우
    text = re.sub(r'\b(\w)\s+\1\b', r'\1', text)
    
    return text


# stopwords + 반복어구 제거
def text_preprocess(text):
    text = remove_stopwords(text)
    return text

In [ ]:
import json
from tqdm import tqdm

def make_clean_data(data_path):
    with open(data_path, "r") as f:
        data = json.load(f)

    data = file_preprocess(data)
    
    text_list = []
    def make_chat(inp):
        chat = ""
        for cvt in inp['conversation']:
            chat += cvt['utterance']

        return text_preprocess(chat)

    for example in tqdm(data):
        text_list.append(make_chat(example['input']))
    
    text = ' '.join(text_list)
    return text

In [ ]:
import re
re.sub(r'name[0-9]\S*', '', "name1이가 그래서 정말 슬펐어")

In [ ]:
from datetime import datetime

# 현재 날짜와 시간 가져오기
now = datetime.now()

# 날짜와 시간을 문자열로 포맷팅
current_time = now.strftime("%Y-%m-%d %H:%M")

print(current_time)

# 화자 1 - 화자 2

꼭 넣어줘야 할까? -> ID 대신 <|A|>, <|B|>로 바꿔서 후처리하자

In [ ]:
import json

# 예시 데이터
data = [
    {   
        "id" : "conv-0001",
        "speaker_ids" : {"SD001": "value1", "SD002": "value2"}
    },
    {   
        "id" : "conv-0002",
        "speaker_ids" : {"SD003": "value1", "SD004": "value2"}
    }
]

# JSON 파일에 데이터를 한 줄씩 추가하는 함수
def save_to_json_file(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 파일 경로
file_path = 'data_test.json'

# 데이터 추가
save_to_json_file(file_path, data)

In [ ]:
from transformers import AutoTokenizer
# tokenizer special token
tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")
tokenizer.pad_token = tokenizer.eos_token
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Add special tokens
special_tokens_dict = {'additional_special_tokens': ['<|A|>', '<|B|>']}
tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
tokenizer.all_special_tokens

# A, B 치환 테스트

In [1]:
# utils.py
import evaluate
import numpy as np
import re
import json
from transformers import EvalPrediction, AutoTokenizer
import torch
import random
from typing import Dict
import argparse

config_name = input("## input utils config file name ##\n")

print("## utils Config File : ", config_name)
# change abspath to run in ipynb file
with open(f'/mnt/g/내 드라이브/국립국어원_일상대화요약/korean_dialog/dialogue-summarization/configs/{config_name}', 'r') as f:
    config = json.load(f)

# with open(f'configs/{config_name}', 'r') as f:
#     config = json.load(f)

rouge = evaluate.load('rouge')
bert_score = evaluate.load('bertscore')
bleurt = evaluate.load('bleurt', 'bleurt-large-512', module_type="metric")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    return preds, labels

def preprocess_logits_for_metrics(logits, labels):
      """
      Original Trainer may have a memory leak. 
      This is a workaround to avoid storing too many tensors that are not needed.
      """
      pred_ids = torch.argmax(logits, dim=-1)
      return pred_ids, labels

def compute_metrics(eval_pred: EvalPrediction):
    tokenizer = AutoTokenizer.from_pretrained(config["arch"]["model_id"])

    # compute Rouge-1 F1 score
    labels = eval_pred.label_ids # (batch_size, seq_len)
    predictions = eval_pred.predictions[0].reshape(labels.shape[0],-1) # (batch_size, seq_len)

    # Replace -100 with pad_token_id
    mask  = np.where(labels == -100)
    labels[mask] = tokenizer.pad_token_id
    predictions[mask] = tokenizer.pad_token_id

    # Decoding
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Simple postprocessing
    predictions, labels = postprocess_text(predictions, labels)

    rouge_scores = rouge.compute(predictions=predictions, references=labels, rouge_types=["rouge1"])
    # rouge_scores = rouge.get_scores(predictions, labels, avg=True)
    bert_scores = bert_score.compute(predictions=predictions, references=labels, lang="ko")
    bleurt_scores = bleurt.compute(predictions=predictions, references=labels)

    bertScore = sum(bert_scores['f1']) / len(labels)
    bleurtScore = sum(bleurt_scores['scores']) / len(labels)

    rouge1 = rouge_scores['rouge1']
    total = (bertScore + bleurtScore + rouge1) / 3

    return {"total" : round(total, 4), "rouge1" : round(rouge1, 4), "BERTScore" : round(bertScore, 4), "BLEURT": round(bleurtScore, 4)}


## Preprocess functions ##
def remove_empty_utterance(data:json):
    """
    Remove empty utterances from the data
    """
    for example in data:
        example['input']['conversation'] = [cvt for cvt in example['input']['conversation'] if cvt['utterance'] != '']
    return data


def correct_wrong_output(data:json):
    """
    Correct wrong speakers in outputs of train samples 'train-000401', 'train-000402'
    """
    data[400]['output'] = data[400]['output'].replace('SD2100504','SD2110504')
    data[401]['output'] = data[401]['output'].replace('SD2110503','SD2100503')

    return data


def file_preprocess(data:json, is_train=False):
    data = remove_empty_utterance(data)

    if is_train == True:
        data = correct_wrong_output(data)

    return data


"""
불용어 처리

## hyperstella2 ##
- name1, name2..
- 뒤에 물결이 붙는 경우 ("음~", "아~")
- 그, 뭐, 어, 인제, 막, 아, 음, 읍, 오, 으
- 한 글자가 두번 이상 반복되는 경우 ("또 또", "그 그")


## nova ##
- name 그대로 유지
- 뒤에 물결이 붙는 경우 ("음~", "아~")
- 그, 뭐, 어, 인제, 막, 아, 음, 읍, 오, 으
- 단어가 두 번 이상 반복되는 경우 제거 ( r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b')


## nova3, hypernova ##
- name 그대로 유지
- 뒤에 물결이 붙는 경우 ("음~", "아~")
- 그, 뭐, 어, 인제, 막, 아, 음, 읍, 오, 으
- 단어가 두 번 이상 반복되는 경우 제거 ( r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b')
- x를 포함한 단어 제거 (r'\b[가-힣a-zA-Z]*[xX][가-힣a-zA-Z]*\b')


"""

stopwords_pattern = [r'\w~', r'\b으\b', r'\b그\b', r'\b뭐\b', r'\b어\b',  r'\b인제\b', r'\b이제\b', r'\b막\b', r'\b아\b', r'\b음\b', r'\b읍\b', r'\b오\b', r'\b으\b'] # r'name[0-9]\S*'

def remove_stopwords(text):
    # 커스텀 불용어 제거
    for pattern in stopwords_pattern:
        text = re.sub(pattern, '', text)
    
    # x를 포함한 단어 제거
    text = re.sub(r'\b[가-힣a-zA-Z]*[xX][가-힣a-zA-Z]*\b', '', text)

    # 단어가 두 번 이상 반복되는 경우 -> 1개로
    # text = re.sub(r'\b(\w)\s+\1\b', r'\1', text)
    text = re.sub(r'\b([가-힣a-zA-Z0-9_]+)\s+\1\b', r'\1', text)

    # 공백 두 번 이상 연속 -> 1개로
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

# stopwords + 반복 어구 제거
def text_preprocess(text):
    text = remove_stopwords(text)
    
    return text



def set_seed(config: Dict):
    seed = config['seed']
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

# argparse에서 boolean인자 받기
def str2bool(v):
    if isinstance(v, bool):
       return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

/home/hwyewon/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-31 03:54:05.546469: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-31 03:54:05.678581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 03:54:05.727427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 03:54:05.741228: E 

## utils Config File :  config_bllossom.json
INFO:tensorflow:Reading checkpoint /home/hwyewon/.cache/huggingface/metrics/bleurt/bleurt-large-512/downloads/extracted/8bfa7908c0a12eb1d407684a0b23baa967ad9775d360bc1603b9295f68c33339/bleurt-large-512.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


I0000 00:00:1722365661.046996    1632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722365661.053077    1632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722365661.053110    1632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722365661.055577    1632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1722365661.055628    1632 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

INFO:tensorflow:BLEURT initialized.


INFO:tensorflow:BLEURT initialized.


In [2]:
# data.py
import json
import os
import torch
from torch.utils.data import Dataset

# JSON 파일에 데이터를 한 줄씩 추가하는 함수
def save_to_json_file(file_path, data):
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


class CustomDataset(Dataset):
    def __init__(self, fname, tokenizer):
        IGNORE_INDEX=-100
        self.inp = []
        self.label = []

        PROMPT = '''당신은 유능한 AI 어시스턴트 입니다. [대화 내용]과 [대화 키워드]를 보고, [요약문]을 생성해주세요.\n'''

        with open(fname, "r") as f:
            data = json.load(f)

        if fname.split('/')[-1].split('.')[0].split('_')[1] == "train":
            data = file_preprocess(data, is_train=True)
        else:
            data = file_preprocess(data, is_train=False)

        ID_FILE = []

        def make_chat(id, inp):
            chat = [f"[대화 키워드]\n{', '.join(inp['subject_keyword'])}에 대한 대화 내용입니다.\n[대화 내용]"]
            
            # json row로 저장
            # 먼저 나온 speaker를 A로 할당
            id_row = {"id" : id, "speaker_ids" : {inp['conversation'][0]['speaker'] : "<|A|>"}}

            for cvt in inp['conversation']:
                speaker_idx = cvt['speaker']

                # 2번째 발화자 추가 : 뒤에 나온 speaker를 B로 할당
                if speaker_idx not in id_row["speaker_ids"].keys():
                    id_row["speaker_ids"][speaker_idx] = "<|B|>"

                utterance = text_preprocess(cvt['utterance'])

                # 비어있는 문장 제거
                if len(utterance) == 0:
                    continue

                chat.append(f"{id_row['speaker_ids'][speaker_idx]}: {utterance}")
                
                

            chat = "\n".join(chat)

            # speaker dict를 json 파일에 저장
            ID_FILE.append(id_row)

            question = f"[요약문]\n"
            chat = chat + "\n\n" + question

            return chat
        
        for example in data:
            chat = make_chat(example["id"], example["input"])
            message = [
                {"role": "system", "content": PROMPT},
                {"role": "user", "content": chat},
            ]
     
            source = tokenizer.apply_chat_template(
                message,
                add_generation_prompt=True,
                return_tensors="pt",
            )

            target = example["output"]
            if target != "":
                target += tokenizer.eos_token
            target = tokenizer(target,
                      return_attention_mask=False,
                      add_special_tokens=False,
                      return_tensors="pt")
            target["input_ids"] = target["input_ids"].type(torch.int64)

            input_ids = torch.concat((source[0], target["input_ids"][0]))
            labels = torch.concat((torch.LongTensor([IGNORE_INDEX] * source[0].shape[0]), target["input_ids"][0]))
            self.inp.append(input_ids)
            self.label.append(labels)
        
        save_to_json_file(os.path.join("/mnt/g/내 드라이브/국립국어원_일상대화요약/korean_dialog/dialogue-summarization/resource/data/", f"ID_{fname.split('/')[-1]}"), ID_FILE)

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx]


class DataCollatorForSupervisedDataset(object):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, instances):
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            [torch.tensor(ids) for ids in input_ids], batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence([torch.tensor(lbls) for lbls in labels], batch_first=True, padding_value=-100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [3]:
tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")
tokenizer.pad_token = tokenizer.eos_token
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Add special tokens
special_tokens_dict = {'additional_special_tokens': ['<|A|>', '<|B|>']}
tokenizer.add_special_tokens(special_tokens_dict)

dataset = CustomDataset("/mnt/g/내 드라이브/국립국어원_일상대화요약/korean_dialog/dialogue-summarization/resource/data/일상대화요약_dev.json", tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
tokenizer.decode(dataset[0])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n당신은 유능한 AI 어시스턴트 입니다. [대화 내용]과 [대화 키워드]를 보고, [요약문]을 생성해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n[대화 키워드]\n식품, 건강, 검사에 대한 대화 내용입니다.\n[대화 내용]\n<|A|>: 근데 건강이 우리만 건강한다고 되는 게 아니잖아. 애들도 건강도 챙겨줘야 되고 그러잖아. 그러니까 너는 애들한테 따로 먹이는 식품 이런 거 있어?\n<|B|>: 유산 유산균하고 비타민 그런데 애들이 잘 안 먹지. 나도 잘 안 먹는 데 애들이 먹나? 근데 쫓아다니면서 챙겨줄 수도 없고 그런데 유산균은 꼭 먹이라 그러더라고. 유산균 장이 건강해야지 전체적으로 다 좋아진다고 그러더라고.\n<|B|>: 그래서 될 수 있으면 비타민도 사놓기는 했는데 그거는 넘어가더라도 나는 유산균은 꼭 먹으라고 얘기한다. 유산균이 어떤 사람은 공복에 먹으라 그러고 어떤 사람은 그냥 아무 때나 먹으라고 하는데 모르겠어. 언제 먹는 게 좋은지\n<|B|>: 그래서 name2이는 또 천식에 아토피까지 있잖아. 진짜 우리 name2이는 건강을 누구보다 진짜 걔는 정말 신경 써가지고 지켜야 되는데 지금은 애들 유산균 먹이는 거에 집중하지 유산균이 젤 좋다고는 하더라고. 피부나 아니면 장내 활동이 좋아야지 감기나 이런 것도 걸렸을 때 빨리 낫는다고.\n<|B|>: 그래서 근데 어떤 유산균이 또 좋은지 몰라. 가루로 된 걸 먹이는 사람도 있고 알약으로 된 걸 먹이는 사람도 있는데 나는 그냥 가루로 된 거 그리고 광고 많이 나오는 거 나도 거기에 대한 정보가 많이 없어서 그냥 오래된 회사 거? 이런 데 거 먹이고\n<|B|>: 그래서 애들이 건강해야 나도 편하기는 한데 애들 건강까지 챙겨준다는 것도 어렵고 지금은 유산균만 먹여. 유산균 먹이고 야채 많이 먹이고 그러는데 야채도 잎채소랑 줄기랑 또 

# BART 요약 테스트

요약해서 input length를 줄인다면?